In [6]:
import sys
import os
import numpy as np
from datetime import datetime
import glob
#Get the path of the current file
current_path = os.getcwd()
# Add the evoman_framework folder to sys.path
sys.path.append(os.path.join(current_path, "evoman"))
# Now you can import contents from the evoman environment
from evoman.environment import Environment
from evoman.controller import Controller
# Initialize environment

# Set headless mode to improve running speed
headless = True
if headless:
    os.environ["SDL_VIDEODRIVER"] = "dummy"

experiment_name = "multi_island_optimization"
test_log_path = 'multi_island_optimization/test_logs/'
module_path = 'multi_island_optimization/models/' 
if not os.path.exists(experiment_name):
    os.makedirs(experiment_name)

if not os.path.exists(test_log_path):
    os.makedirs(test_log_path)

if not os.path.exists(module_path):
    os.makedirs(module_path)
    
# Define Sigmoid activation function
def sigmoid_activation(x):
    return 1.0 / (1.0 + np.exp(-x))
# Custom controller with a simple neural network structure
class player_controller(Controller):
    def __init__(self, _n_hidden):
        self.n_hidden = [_n_hidden]  # Number of neurons in the hidden layer

    def set(self, controller, n_inputs):
        if self.n_hidden[0] > 0:
            # Extract weights and biases from controller parameters to build the neural network
            self.bias1 = controller[: self.n_hidden[0]].reshape(1, self.n_hidden[0])
            weights1_slice = n_inputs * self.n_hidden[0] + self.n_hidden[0]
            self.weights1 = controller[self.n_hidden[0] : weights1_slice].reshape(
                (n_inputs, self.n_hidden[0])
            )
            self.bias2 = controller[weights1_slice : weights1_slice + 5].reshape(1, 5)
            self.weights2 = controller[weights1_slice + 5 :].reshape(
                (self.n_hidden[0], 5)
            )

    def control(self, inputs, controller):
        # Normalize inputs
        inputs = (inputs - min(inputs)) / float((max(inputs) - min(inputs)))
        if self.n_hidden[0] > 0:
            output1 = sigmoid_activation(inputs.dot(self.weights1) + self.bias1)
            output = sigmoid_activation(output1.dot(self.weights2) + self.bias2)[0]
        else:
            bias = controller[:5].reshape(1, 5)
            weights = controller[5:].reshape((len(inputs), 5))
            output = sigmoid_activation(inputs.dot(weights) + bias)[0]

        # Determine player actions based on output
        left = 1 if output[0] > 0.5 else 0
        right = 1 if output[1] > 0.5 else 0
        jump = 1 if output[2] > 0.5 else 0
        shoot = 1 if output[3] > 0.5 else 0
        release = 1 if output[4] > 0.5 else 0

        return [left, right, jump, shoot, release]


# Initialize controller
player_controller_instance = player_controller(10)  # Example with 10 hidden neurons



def initialize_islands(enemylist):
    env.enemies = enemylist
    env.multiplemode="yes"
    islands = []
    for _ in range(n_islands):
        #生成一个二维数组pop，维度分别为个体数目，以及特征数目
        pop = np.random.uniform(dom_l, dom_u, (npop // n_islands, n_vars))
        fit_pop = evaluate(pop)
        islands.append((pop, fit_pop))
        #一个元组列表
    return islands


def migrate(islands):
    num_migrants = int(migration_rate * (npop // n_islands))
    for i in range(n_islands):
        # Select migrants from island i
        migrants = islands[i][0][:num_migrants]
        # Migrate to the next island (circular migration)
        target_island = (i + 1) % n_islands
        islands[target_island][0][-num_migrants:] = migrants
    return islands


# Uniform crossover
def uniform_crossover(p1, p2):
    offspring = np.zeros_like(p1)
    for i in range(len(p1)):
        if np.random.rand() < 0.5:
            offspring[i] = p1[i]
        else:
            offspring[i] = p2[i]
    return offspring


# Fixed mutation
def mutation_operation(offspring):
    for i in range(len(offspring)):
        if np.random.uniform(0, 1) <= mutation:
            offspring[i] += np.random.normal(0, 1)
    return offspring


# Elitism selection
def elitism(pop, fit_pop, elite_size):
    elite_indices = np.argsort(fit_pop)[-elite_size:]
    elite_pop = pop[elite_indices]
    elite_fitness = fit_pop[elite_indices]
    return elite_pop, elite_fitness


# Crossover and mutation operations
def crossover_and_mutation(pop):
    total_offspring = np.zeros((0, n_vars))
    for p in range(0, pop.shape[0], 2):
        p1 = pop[np.random.randint(0, pop.shape[0])]
        p2 = pop[np.random.randint(0, pop.shape[0])]

        # Uniform crossover
        offspring = uniform_crossover(p1, p2)

        # Fixed mutation operation
        offspring = mutation_operation(offspring)

        # Fitness constraint
        offspring = np.clip(offspring, dom_l, dom_u)
        total_offspring = np.vstack((total_offspring, offspring))

    return total_offspring 

def evaluate(x):
    #把每一列的权重进行测试
    return np.array([simulation(env, ind) for ind in x])

def simulation(env, x):
    env.player_controller.set(x, n_inputs)  # Set controller parameters
    f, _, _, _ = env.play(pcont=x)
    return f


def evolve_islands(islands,enemylist):
    for i in range(n_islands):
        
        env.enemies = [enemylist[i]]
        env.multiplemode="no"
        pop, fit_pop = islands[i]
        elite_size = int(elitism_rate * len(pop))
        elite_pop, elite_fitness = elitism(pop, fit_pop, elite_size)

        # Crossover and mutation
        offspring = crossover_and_mutation(pop)
        
        env.enemies = enemylist
        env.multiplemode="yes"
        fit_offspring = evaluate(offspring)

        # Combine elites and offspring
        pop = np.vstack((elite_pop, offspring[: len(pop) - elite_size]))
        fit_pop = np.hstack((elite_fitness, fit_offspring[: len(pop) - elite_size]))

        # Update the island's population and fitness
        islands[i] = (pop, fit_pop)
    return islands

# Run evolution process
def run_evolution(enemylist):
    current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{experiment_name}/test_logs/MI_log_{current_time}.txt"

    with open(filename, "a") as file:
        file.write(
            f"npop{npop}_gens{gens}_"
            f"enemy{enemylist}_mut{mutation:.2f}_elitism{elitism_rate:.2f}_{current_time}\n"
        )
        file.write("gen best mean std\n")

        islands = initialize_islands(enemylist)

        for generation in range(gens):
            
            
            # Evolve each island
            islands = evolve_islands(islands,enemylist)

            
            
            
            # Perform migration every migration_interval generations
            if generation % migration_interval == 0 and generation > 0:
                islands = migrate(islands)

            
            # Record results (best, mean, std across all islands)
            all_fitness = np.hstack([fit_pop for _, fit_pop in islands])
            best = np.max(all_fitness)
            mean = np.mean(all_fitness)
            std = np.std(all_fitness)

            result = f"{generation} {best:.6f} {mean:.6f} {std:.6f}"
            print(result)
            file.write(result + "\n")

        # Find the best solution across all islands
        all_pop = np.vstack([pop for pop, _ in islands])
        all_fit_pop = np.hstack([fit_pop for _, fit_pop in islands])
        save_final_solution(all_pop, all_fit_pop, experiment_name, env.enemies, npop, gens, mutation, elitism_rate)

def save_final_solution(
    pop, fit_pop, experiment_name, enemies, npop, gens, mutation, elitism_rate
):
    best_index = np.argmax(fit_pop)
    best_solution = pop[best_index]
    best_fitness = fit_pop[best_index]

    # Get current timestamp
    current_time = datetime.now().strftime("%Y%m%d_%H%M%S")

    # Filename uses the best fitness
    filename = (
        f"{experiment_name}/models/MIModel_fitness{best_fitness:.6f}_npop{npop}_gens{gens}_"
        f"enemy{enemies}_mut{mutation:.2f}_elitism{elitism_rate:.2f}_{current_time}.txt"
    )

    # Save the best individual's controller weights and biases
    np.savetxt(filename, best_solution)

    # Print information about the saved file
    print(f"Best solution saved as {filename} with fitness: {best_fitness:.6f}")
 
    
# Load best solution
def load_best_solution(filepath):
    return np.loadtxt(filepath)

# Simulation test
def simulation_test(env, x):
    env.player_controller.set(x, n_inputs)  # Set controller parameters
    f, player_life, enemy_life, _ = env.play(pcont=x)  # Return player and enemy life
    return f, player_life, enemy_life
    


In [3]:
env = Environment(
    experiment_name=experiment_name,
    #multiplemode="yes",
    playermode="ai",
    player_controller=player_controller_instance,
    enemymode="static",
    level=2,
    speed="fastest",
    visuals=False,
)
#set your enemy here
enemylist = [5,6,7,8]
 
# Genetic algorithm parameters
n_inputs = 20  # Number of inputs to the controller
n_vars = (
    n_inputs * 10 + 10 + 10 * 5 + 5
)  # Total number of weights and biases in the controller
dom_u = 1
dom_l = -1
npop = 120  # Population size
gens = 500  # Total generations
mutation = 0.2  # Mutation rate
elitism_rate = 0.2  # Elitism rate
migration_interval = 5  # Every 10 generations
migration_rate = 0.2 # 10% of the population migrates
n_islands = 4
 
for _ in range(10):
    run_evolution(enemylist)


shm_open() failed: No such file or directory
ALSA lib conf.c:4028:(snd_config_hooks_call) Cannot open shared library libasound_module_conf_pulse.so (/home/taoyida/miniconda3/envs/EC/lib/alsa-lib/libasound_module_conf_pulse.so: cannot open shared object file: No such file or directory)
ALSA lib pcm.c:2722:(snd_pcm_open_noupdate) Unknown PCM default



MESSAGE: Pygame initialized for simulation.
0 44.637537 -1.744583 9.441187
1 44.637537 0.821956 11.457430
2 44.637537 2.565993 12.914314
3 44.637537 3.901950 13.689944
4 44.637537 1.213735 14.318561
5 44.637537 6.319791 16.310668
6 44.637537 4.778995 13.987610
7 46.841068 10.066563 18.911554
8 46.841068 5.487511 16.932579
9 62.958796 10.736721 22.419461
10 62.958796 6.491719 19.610971
11 62.958796 13.696993 22.238474
12 62.958796 9.100905 19.659221
13 62.958796 11.032957 22.398419
14 62.958796 6.355426 20.068802
15 62.958796 7.100212 24.160318
16 62.958796 6.551469 20.931761
17 62.958796 11.620768 22.658783
18 62.958796 7.098765 20.405263
19 62.958796 7.798474 23.398700
20 62.958796 3.931018 20.377128
21 62.958796 11.523689 23.107176
22 62.958796 6.050347 20.456329
23 62.958796 10.391275 24.014217
24 62.958796 6.184725 20.643684
25 62.958796 11.907861 22.727819
26 62.958796 7.761790 20.173447
27 64.344362 17.022706 25.806200
28 64.344362 15.554816 24.220102
29 64.344362 14.691468 23.9

测试

In [44]:
def test_loaded_model():
    # Hardcoded model file path
    filepath = f"{experiment_name}/models/MIModel_fitness92.937405_npop100_gens50_enemy[4]_mut0.20_elitism0.20_20241019_203405.txt"

    fitness_scores = []
    gains = []  # Used to store individual_gain for each run

    # Load the saved model
    solution = load_best_solution(filepath)

    # Run 5 times
    for i in range(5):
        fitness, player_life, enemy_life = simulation_test(env, solution)
        fitness_scores.append(fitness)

        # Calculate individual_gain
        individual_gain = player_life - enemy_life
        gains.append(individual_gain)

        print(f"Run {i+1} fitness: {fitness}, individual_gain: {individual_gain}")

    # Calculate the average fitness
    average_fitness = np.mean(fitness_scores)
    average_gain = np.mean(gains)

    print(f"\nAverage fitness over 5 runs: {average_fitness}")
    print(f"Average individual gain over 5 runs: {average_gain}")


# # Run test
# test_loaded_model()

In [7]:
import glob
import numpy as np
import csv
import os

def evaluate_models_to_csv(experiment_name, enemy):
    # Get all model files
    model_files = glob.glob(f"{experiment_name}/models/E{enemy}1020/*.txt")
    print(f"Found model files: {model_files}")  # Debugging line

    # Check if there are any model files
    if not model_files:
        raise FileNotFoundError(
            f"No model files found in {experiment_name}/models/E{enemy}/"
        )

    output_csv = f"{experiment_name}_enemy{enemy}.csv"  # Output CSV file

    results = []

    for filepath in model_files:
        # Load the saved model
        solution = load_best_solution(filepath)
        print(f"Loaded solution from {filepath}")  # Debugging line

        # Run 1 time
        fitness, player_life, enemy_life = simulation_test(env, solution)
        print(f"Test results - Fitness: {fitness}, Player Life: {player_life}, Enemy Life: {enemy_life}")  # Debugging line

        # Calculate individual_gain
        individual_gain = player_life - enemy_life

        print(
            f"Model: {filepath}, fitness: {fitness}, individual_gain: {individual_gain}"
        )

        # Store the result
        results.append([filepath, fitness, individual_gain])

    # Write results to CSV
    with open(output_csv, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["Model", "Fitness", "Individual Gain"])
        writer.writerows(results)

    print(f"Results have been written to {output_csv}")

In [10]:
env.enemies=[1,2,3,4]
env.multiplemode = 'yes'
evaluate_models_to_csv(experiment_name, 1234)

Found model files: ['multi_island_optimization/models/E12341020/MIModel_fitness46.946123_npop120_gens500_enemy[1, 2, 3, 4]_mut0.20_elitism0.20_20241020_141343.txt', 'multi_island_optimization/models/E12341020/MIModel_fitness38.649025_npop120_gens500_enemy[1, 2, 3, 4]_mut0.20_elitism0.20_20241020_143231.txt', 'multi_island_optimization/models/E12341020/MIModel_fitness51.166553_npop120_gens500_enemy[1, 2, 3, 4]_mut0.20_elitism0.20_20241020_143654.txt', 'multi_island_optimization/models/E12341020/MIModel_fitness64.192672_npop120_gens500_enemy[1, 2, 3, 4]_mut0.20_elitism0.20_20241020_141832.txt', 'multi_island_optimization/models/E12341020/MIModel_fitness47.874066_npop120_gens500_enemy[1, 2, 3, 4]_mut0.20_elitism0.20_20241020_140838.txt', 'multi_island_optimization/models/E12341020/MIModel_fitness33.580143_npop120_gens500_enemy[1, 2, 3, 4]_mut0.20_elitism0.20_20241020_140419.txt', 'multi_island_optimization/models/E12341020/MIModel_fitness38.643935_npop120_gens500_enemy[1, 2, 3, 4]_mut0.20